In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
             .appName("myapp.com") \
             .getOrCreate()

In [8]:
import pyspark

In [9]:
df = spark.read.json(r'C:\Users\91810\Downloads\Grocery_and_Gourmet_Food_5.json.gz')

In [10]:
from pyspark.sql.functions import *

In [11]:
df.show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|4639725183| null|    5.0| No adverse comment.|11 19, 2014|A1QVBUH9E1V6I8|   Jamshed Mathur| null|          Five Stars|    1416355200|    true|null|
|4639725183| null|    5.0|Gift for college ...|10 13, 2016|A3GEOILWLK86XM|        itsjustme| null|      Great product.|    1476316800|    true|null|
|4639725183| null|    5.0|If you like stron...|11 21, 2015|A32RD6L701BIGP|  Krystal Clifton| null|              Strong|    1448064000|    true|null|
|4639725183| null|    5.0|Love the tea. The...|08 12, 2015|A2UY1O1FBGKIE6|          U. Kane| null|        

In [12]:
df_min = df.agg({'overall':'min'})
df_min = df_min.withColumnRenamed('min(overall)','Min_rating')

In [13]:
df_min.show()

+----------+
|Min_rating|
+----------+
|       1.0|
+----------+



In [14]:
df.filter(df.overall == df_min.first().Min_rating).show()

+----------+--------------------+-------+--------------------+-----------+--------------+------------------+--------------------+--------------------+--------------+--------+----+
|      asin|               image|overall|          reviewText| reviewTime|    reviewerID|      reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+--------------------+-------+--------------------+-----------+--------------+------------------+--------------------+--------------------+--------------+--------+----+
|4639725043|                null|    1.0|This tea looks li...|03 11, 2016|A2PMV21JFDKJIU|       shopAholicr|                null|does not look any...|    1457654400|    true|null|
|4639725043|                null|    1.0|I bought this bec...|04 30, 2013|A2TURHWPQ343WP|                KS|                null|        Disappointed|    1367280000|    true|null|
|9742356831|                null|    1.0|Did not say that ...|07 11, 2014| A9BB1RGFI8PP8|        Sus

In [15]:
df_max = df.agg({'overall':'max'})
df_max = df_max.withColumnRenamed('max(overall)','Max_rating')

In [16]:
df_max.show()

+----------+
|Max_rating|
+----------+
|       5.0|
+----------+



In [17]:
df.filter(df.overall == df_max.first().Max_rating).show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|4639725183| null|    5.0| No adverse comment.|11 19, 2014|A1QVBUH9E1V6I8|   Jamshed Mathur| null|          Five Stars|    1416355200|    true|null|
|4639725183| null|    5.0|Gift for college ...|10 13, 2016|A3GEOILWLK86XM|        itsjustme| null|      Great product.|    1476316800|    true|null|
|4639725183| null|    5.0|If you like stron...|11 21, 2015|A32RD6L701BIGP|  Krystal Clifton| null|              Strong|    1448064000|    true|null|
|4639725183| null|    5.0|Love the tea. The...|08 12, 2015|A2UY1O1FBGKIE6|          U. Kane| null|        

In [18]:
df_max_length = df.withColumn("len_Description",length(col("reviewText"))).groupBy().max("len_Description")

In [19]:
df_max_length = df_max_length.withColumnRenamed('max(len_Description)','Max_Length')

In [20]:
df_max_length.first().Max_Length

19057

In [21]:
df = df.withColumn('text_length',length(col('reviewText')))
df = df.filter(df.text_length.isNotNull())
df_final = df.filter(df.text_length == df_max_length.first().Max_Length)


In [22]:
df_final.show()

+----------+-----+-------+--------------------+-----------+-------------+------------+-----+--------------------+--------------+--------+----+-----------+
|      asin|image|overall|          reviewText| reviewTime|   reviewerID|reviewerName|style|             summary|unixReviewTime|verified|vote|text_length|
+----------+-----+-------+--------------------+-----------+-------------+------------+-----+--------------------+--------------+--------+----+-----------+
|B001DEX99Q| null|    2.0|I am a sweet-ahol...|10 15, 2009|AJNJNA093JUW0|Serendipity.| null|The Real Review o...|    1255564800|    true|  95|      19057|
+----------+-----+-------+--------------------+-----------+-------------+------------+-----+--------------------+--------------+--------+----+-----------+



In [23]:
df.first().unixReviewTime

1416355200

In [24]:
df.withColumn('date_formatted',date_format(lit(from_unixtime(col("unixReviewTime")).cast('date')),'MM-dd-yyyy')).show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+-----------+--------------+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|style|             summary|unixReviewTime|verified|vote|text_length|date_formatted|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+-----------+--------------+
|4639725183| null|    5.0| No adverse comment.|11 19, 2014|A1QVBUH9E1V6I8|   Jamshed Mathur| null|          Five Stars|    1416355200|    true|null|         19|    11-19-2014|
|4639725183| null|    5.0|Gift for college ...|10 13, 2016|A3GEOILWLK86XM|        itsjustme| null|      Great product.|    1476316800|    true|null|         25|    10-13-2016|
|4639725183| null|    5.0|If you like stron...|11 21, 2015|A32RD6L701BIGP|  Krystal Clifton| null|              Strong| 

In [38]:
df.write.mode('append').parquet("/tmp/output/people.parquet")